# Pact-Karate demo
## Setup
Build Docker containers for Karate, Pact-to-Karate

In [14]:
# Let's start out by building a Docker container image to run Karate
!docker rmi --force karate-docker:latest
!docker build -t karate-docker ~/Projects/karate-docker/

Untagged: karate-docker:latest
Deleted: sha256:503b7af0f75ea5d24a90a0711ba3fd981a0f6d66ca3174f12b6a0fd7c74ed0cf
Deleted: sha256:26cfdea1ff915f5d22f39432c999993ad0db84873b4b404ccc37543bb7d2036d
Deleted: sha256:d6cff3e058edd79a218e617947b14dd5a57a77decdd7dcb18d24ea8abde2311a
Sending build context to Docker daemon  26.14MB
Step 1/8 : FROM openjdk:12-alpine
 ---> 0c68e7c5b7a0
Step 2/8 : ENV KARATE_VERSION 0.9.4
 ---> Using cache
 ---> 1b9d9d557e65
Step 3/8 : WORKDIR /
 ---> Using cache
 ---> ad0a3db010c7
Step 4/8 : RUN wget -O karate.jar https://github.com/intuit/karate/releases/download/v$KARATE_VERSION/karate-$KARATE_VERSION.jar
 ---> Using cache
 ---> d4ac2dd36132
Step 5/8 : COPY features/ features/
 ---> Using cache
 ---> b4e892e16437
Step 6/8 : COPY results/ results/
 ---> a7141b1d438e
Step 7/8 : EXPOSE 8999
 ---> Running in 035bbc1558f8
Removing intermediate container 035bbc1558f8
 ---> 62d9bf25a7b6
Step 8/8 : ENTRYPOINT if [ -z "$MOCK" ]; then /opt/openjdk-12/bin/java -jar karate.ja

In [16]:
# Now let's build a Docker container whose sole purpose is to convert Pact contracts into Karate stubs & test cases
!docker rmi --force pact-karate:latest
!docker build -t pact-karate ~/Projects/pact-to-karate/golang/src

Untagged: pact-karate:latest
Deleted: sha256:a505b93950fdf6a2972a650ec6977b42fb05f3192e96a31f4b1910fe03b6d475
Deleted: sha256:faf8cb12c4365a37c15adf2650acfb2677ed16c5d2be3d714d728a2d2457e7e2
Sending build context to Docker daemon  12.29kB
Step 1/9 : FROM golang:alpine AS builder
 ---> d4953956cf1e
Step 2/9 : RUN apk update && apk add --no-cache git
 ---> Using cache
 ---> d08d5dbfaa5b
Step 3/9 : WORKDIR $GOPATH/src/mypackage/myapp/
 ---> Using cache
 ---> 83dc5a4d0a0a
Step 4/9 : COPY . .
 ---> Using cache
 ---> 31c1a23db9dc
Step 5/9 : RUN go get -d -v
 ---> Using cache
 ---> c5cc7e090372
Step 6/9 : RUN CGO_ENABLED=0 GOOS=linux go build -o /go/bin/main
 ---> Using cache
 ---> f83f07862abc
Step 7/9 : FROM scratch
 ---> 
Step 8/9 : COPY --from=builder /go/bin/main /go/bin/main
 ---> 9e4691667bf6
Step 9/9 : ENTRYPOINT ["/go/bin/main"]
 ---> Running in 3b3b4966ef61
Removing intermediate container 3b3b4966ef61
 ---> 98683e77d632
Successfully built 98683e77d632
Successfully tagged pact-karate

In [17]:
# Note that the pact-karate image is tiny - all it contains is the executable file within a scratch container. 
# However the karate-docker container requires a full JVM plus the Karate JAR file - it's much larger
!docker images | grep karate-docker
!docker images | grep pact-karate

karate-docker                                      latest              f1c0eca9221e        26 seconds ago      364MB
pact-karate                                        latest              98683e77d632        3 seconds ago       2.63MB


In [4]:
# Now go to the directory containing some Pact contracts
!cd ~/Projects/pact-to-karate/pacts

In [5]:
# Starting from a Pact JSON file, we're going to generate a set of Karate stubs & test cases
!cat ~/Projects/pact-to-karate/pacts/sample-pact-v2.json | jq '.'

{
  "consumer": {
    "name": "billy"
  },
  "provider": {
    "name": "bobby"
  },
  "interactions": [
    {
      "description": "My test",
      "providerState": "User billy exists",
      "request": {
        "method": "POST",
        "path": "/users/login",
        "headers": {
          "Content-Type": "application/json"
        },
        "body": {
          "username": "billy",
          "password": "issilly"
        }
      },
      "response": {
        "status": 200
      }
    }
  ],
  "metadata": {
    "pactSpecification": {
      "version": "2.0.0"
    }
  }
}


In [6]:
# Note that the .interactions[].request.path has no server name - we'll have to prepend our stub's address to it when we generate test cases
!cat ~/Projects/pact-to-karate/pacts/sample-pact-v2.json | \
    jq '.interactions[].request.path = "http://localhost:8080\(.interactions[].request.path)"'

{
  "consumer": {
    "name": "billy"
  },
  "provider": {
    "name": "bobby"
  },
  "interactions": [
    {
      "description": "My test",
      "providerState": "User billy exists",
      "request": {
        "method": "POST",
        "path": "http://localhost:8080/users/login",
        "headers": {
          "Content-Type": "application/json"
        },
        "body": {
          "username": "billy",
          "password": "issilly"
        }
      },
      "response": {
        "status": 200
      }
    }
  ],
  "metadata": {
    "pactSpecification": {
      "version": "2.0.0"
    }
  }
}


In [7]:
# Now we're going to generate a stub from that Pact contract, save it to a file, then print the file
!cat ~/Projects/pact-to-karate/pacts/sample-pact-v2.json \
  | docker run -e STUBS=1 -i pact-karate:latest > /tmp/karate.stub.mock.feature
!cat /tmp/karate.stub.mock.feature

Feature: Provider 'bobby' responding to consumer 'billy'

Background:
  * configure cors = true

# My test
Scenario: pathMatches('/users/login') && methodIs('POST') && headerContains('Content-Type', 'application/json') && request["username"] == "billy" && request["password"] == "issilly"

    * def responseStatus = 200

# No match found - default scenario is to return a 404
Scenario:
    * def responseStatus = 404


In [8]:
# Now do the same, but generate Karate test cases instead of stubs
!cat ~/Projects/pact-to-karate/pacts/sample-pact-v2.json | \
    jq '.interactions[].request.path = "http://localhost:8080\(.interactions[].request.path)"'| \
    docker run -e TESTS=1 -i pact-karate:latest > /tmp/karate.tests.feature
!cat /tmp/karate.tests.feature

Feature: Consumer 'billy' sending requests to provider 'bobby'

  Scenario: My test
    Given URL 'http://localhost:8080/users/login'
    And request {"password":"issilly","username":"billy"}
    * def reqHeaders = {"Content-Type":"application/json"}
    * headers reqHeaders
    When method POST
    Then status 200


In [9]:
# Now let's fire up the stub inside a Karate Docker container
!docker run -e MOCK=1 -v "/tmp:/features" --network="host" -p 8999:8080 karate-docker:latest

07:41:44.427 [main] INFO  com.intuit.karate.Main - Karate version: 0.9.4
07:41:45.433 [ForkJoinPool-1-worker-3] WARN  com.intuit.karate - skipping bootstrap configuration: could not find or read file: classpath:karate-config.js
07:41:45.487 [ForkJoinPool-1-worker-3] WARN  com.intuit.karate - skipping bootstrap configuration: could not find or read file: classpath:karate-config.js
07:41:45.676 [pool-1-thread-1] INFO  com.intuit.karate.Runner - <<pass>> feature 1 of 1: features/karate.stub.mock.feature
---------------------------------------------------------
feature: features/karate.stub.mock.feature
report: target/surefire-reports/features.karate.stub.mock.json
scenarios:  2 | passed:  2 | failed:  0 | time: 0.0274
---------------------------------------------------------
Karate version: 0.9.4
elapsed:   1.14 | threads:    1 | thread time: 0.03 
features:     1 | ignored:    0 | efficiency: 0.02
scenarios:    2 | passed:     2 | failed: 0



In [10]:
!docker run -v "/tmp:/features" -v "$(pwd)/results:/results" --network="host" karate-docker:latest

07:41:50.711 [main] INFO  com.intuit.karate.Main - Karate version: 0.9.4
07:41:51.761 [ForkJoinPool-1-worker-3] WARN  com.intuit.karate - skipping bootstrap configuration: could not find or read file: classpath:karate-config.js
07:41:51.804 [ForkJoinPool-1-worker-3] WARN  com.intuit.karate - skipping bootstrap configuration: could not find or read file: classpath:karate-config.js
07:41:51.978 [pool-1-thread-1] INFO  com.intuit.karate.Runner - <<pass>> feature 1 of 4: features/karate-stub.mock.feature
---------------------------------------------------------
feature: features/karate-stub.mock.feature
report: /results/surefire-reports/features.karate-stub.mock.json
scenarios:  2 | passed:  2 | failed:  0 | time: 0.0172
---------------------------------------------------------
07:41:51.996 [ForkJoinPool-1-worker-3] WARN  com.intuit.karate - skipping bootstrap configuration: could not find or read file: classpath:karate-config.js
07:41:52.024 [pool-1-thread-1] INFO  com.intuit.karate.Runne

In [11]:
# Now we can view the test results, which means that CI can pick them apart and 
# work out how to proceed based on whether all tests passed or not
!cat results/surefire-reports/results-json.txt | jq '.'

{
  "features": 4,
  "ignored": 0,
  "efficiency": 0.014350548589341695,
  "failures": {
    "features.karate-tests": "karate-tests.feature:4 - no step-definition method match found for: URL '/users/login'",
    "features.karate.tests": "karate.tests.feature:4 - no step-definition method match found for: URL 'http://localhost:8080/users/login'"
  },
  "totalTime": 18.311300000000003,
  "threads": 1,
  "scenarios": 6,
  "failed": 2,
  "passed": 4,
  "version": "0.9.4",
  "elapsedTime": 1276
}


In [ ]:
!cat ~/Projects/pact-to-karate/pacts/sample-pact-v2.json | jq '.interactions[].request.path = "http://localhost:8080\(.interactions[].request.path)"'